# Markov networks

In this notebook you will know the <b>pgmpy</b> (https://github.com/pgmpy/pgmpy) library for the construction of Markov networks.

Let us start with pgmpy. First of all, we need to import the necessary functions:


In [ ]:
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.models import MarkovModel

## Setting up our model

We will use the classical misconception example, which has the following graph structure:

<img src="images/misconception_graph.png" style="width:200px" />

A MN is composed by an undirected graph structure and the factors. We need to codify both elements.

### Set the structure

First of all, we need to specify that we are constructing a Markov network with a set of undirected edges as follow:

In [ ]:
H = MarkovModel()
H.add_nodes_from(['Alice', 'Bob', 'Charles', 'Debbie'])
H.add_edges_from([('Alice', 'Bob'), 
                  ('Bob', 'Charles'), 
                  ('Charles', 'Debbie'), 
                  ('Debbie', 'Alice')])

print('Check if node `Alice´ is in the graph:','Alice' in H) # check if node in graph
print('Number of nodes in the graph:',len(H))

With this simple code, we specify that there are four variables, each of them representing a student, and that there is an undirected edge connecting <i>Alice</i> and <i>Bob</i>, <i>Bob</i> and <i>Charles</i>, <i>Charles</i> and <i>Debbie</i>, and finally <i>Debbie</i> and <i>Alice</i>.

The corresponding factorization is given by four pairwise factors:

$$p(A,B,C,D)=\frac{1}{\Theta}\phi_{AB}(A,B)\phi_{BC}(B,C)\phi_{CD}(C,D)\phi_{DA}(D,A)$$

### Set up the factor potentials

Once the structure has been defined, we codify the respective factor potentials as follow:

<img src="images/misconception_factors.png" style="height:150px" />


Note that in this case all the random variables are binary.

Firstly, the factor values assigned to all the posible value combinations for <i>Alice</i> and <i>Bob</i> are:


In [ ]:
factorAB = DiscreteFactor(['Alice', 'Bob'], cardinality=[2, 2],
                          values=[30, 5, 1, 10])


Following the same definition the other three factor potentials are:


In [ ]:
factorBC = DiscreteFactor(['Bob', 'Charles'], cardinality=[2, 2],
                          values=[#### YOUR CODE HERE ####]) 

factorCD = DiscreteFactor(['Charles', 'Debbie'], cardinality=[2, 2],
                          values=[#### YOUR CODE HERE ####])

factorDA = DiscreteFactor(['Debbie', 'Alice'], cardinality=[2, 2],
                          values=[#### YOUR CODE HERE ####])


Once the potential factors are defined, we only have to include them into the model:


In [ ]:
H.add_factors(factorAB,factorBC,factorCD,factorDA)


Let us examine our model:


In [ ]:
print('Is the model consistent?',H.check_model())


Another important characteristic of Markov networks is the necessity of a partition function to build a joint probability distribution from the product of factors. This constant, represented in the above equation as $\Theta$, is defined as the sum of the product for all the possible value combinations:
$$\Theta=\sum_{a,b,c,d}\phi_{AB}(A,B)\phi_{BC}(B,C)\phi_{CD}(C,D)\phi_{DA}(D,A)$$

Its value can be obtained as follow:


In [ ]:
print('Partition function value (constant):',H.get_partition_function())

## Using the model

We have already built our model. It is time to use it!

We can want to find the local <b>independencies</b> in the model associated to variable <i>Genetics</i>:


In [ ]:
print('(Conditional) independencies:\n',H.get_local_independencies())


This set is obtained, given each single random variable $X_i$, by looking for all the variables out of the <b>Markov Blanket</b> of $X_i$ as we know that those are independent from $X_i$ given its Mb:

$$X_i\perp \{V \backslash X_i\backslash MB(X_i)\} |MB(X_i)$$

Thus, we might be interested in knowing the Markov Blanket of <i>Alice</i>:

In [ ]:
print('Markov Blanket of `Alice´:',H.markov_blanket('Alice'))


Regarding the set of (conditional) independencies that are encoded by a Markov network and its equivalent, if possible, Bayesian network, we can transform the MN to a BN and observe the produced structure:


In [ ]:
bn = H.to_bayesian_model()
print('Set of directed edges in the BN:\n',bn.edges())


Graphically, it can be drawn as follow:

<img src="images/misconception_bn_imap.png" style="width:200px" />

We can obtain the set of (conditional) independencies of this DAG:

In [ ]:
print('Set of independencies in the BN:\n',bn.get_independencies())


and observe that this DAG $G$ is an I-map of the undirected graph $H$ as $I(G)\subseteq I(H)$.

<hr /> 

## Making inference

Later in this course, we will know different approaches for inference in PGMs. However, let us consider the approach known as <i>Variable Elimination</i> to observe how we can perform different queries.

In [ ]:
from pgmpy.inference import VariableElimination
inference_with_H = VariableElimination(H)


We can obtain the marginal probability distributions of different variables:


In [ ]:
prob_alice = inference_with_H.query(variables = ['Alice'])
print(prob_alice['Alice'])


Similarly, the marginal probability distributions of <i>Bob</i> is:


In [ ]:
prob_bob = #### YOUR CODE HERE ####


It is interesting to observe that, although Alice and Bob are likely to agree according to factor potential $\phi_{AB}$, the marginal probability distributions of both are not. This is a consequence of the rest of the relationships in the model and emphasizes the conceptual distance between potential factor and joint or conditional probability distributions.

The most common use of inference techniques is for propagating the evidence gathered from certain observations. We can calculate the marginal probability <i>Alice</i> and <i>Charles</i> given that <i>Bob</i> is unaware of the misconception:


In [ ]:
prob_Alice_Bob_unaware = inference_with_H.query(variables = ['Alice','Debbie'], 
                                              evidence = {'Bob':0})
print(prob_Alice_Bob_unaware['Alice'])
print(prob_Alice_Bob_unaware['Debbie'])


It is also possible to codify evidence in more than one variable. For example, we can calculate the marginal probability <i>Alice</i> and <i>Debbie</i> given that <i>Bob</i> is unaware of the misconception but <i>Charles</i> is not, which brings more certainty to the model:


In [ ]:
prob_Alice_Bob_disagree_Charles = inference_with_H.query(variables = ['Alice','Debbie'], 
                                              evidence = #### YOUR CODE HERE ####)
print(prob_Alice_Bob_disagree_Charles['Alice'])
print(prob_Alice_Bob_disagree_Charles['Debbie'])


We know from the previous discussion that $D\perp B \mid A,C$. We can experience that: let us imagine that we do know the answer of <i>Alice</i> and <i>Charles</i>:


In [ ]:
prob_Debbie_st_Alice_agree_Charles = inference_with_H.query(variables = ['Debbie'], 
                                                            evidence = #### YOUR CODE HERE ####)
print(prob_Debbie_st_Alice_agree_Charles['Debbie'])


Does any new evidence about the answer of <i>Bob</i> contribute to bring new information to the model?


In [ ]:
# Bob is unaware
prob_Debbie_st_Alice_agree_Charles_indBob0 = inference_with_H.query(variables = ['Debbie'], 
                                                                    evidence = #### YOUR CODE HERE ####)
print(prob_Debbie_st_Alice_agree_Charles_indBob0['Debbie'])

# Bob is aware
prob_Debbie_st_Alice_agree_Charles_indBob1 = inference_with_H.query(variables = ['Debbie'], 
                                                                    evidence = #### YOUR CODE HERE ####)
print(prob_Debbie_st_Alice_agree_Charles_indBob1['Debbie'])

<hr />

## Exercices

- Which is the probability that Charles knows about the misconception given that  is unaware?

- Which is the probability that Charles knows about the misconception given that Bob and Debbie are unaware?

- Which is the probability that Charles knows about the misconception given that the rest of them (Alice, Bob, and Debbie) are unaware?
